# iot:bit als gateway



We willen de Elecfreaks iot:bit gebruiken als iot-gateway voor een lokaal netwerk van micro:bits.

Dit bordje beschikt over een ESP8266-module voor een WiFi-verbinding met het lokale netwerk.
Deze module heeft bovendien HTTP(S) en MQTT-protocollen ingebouwd.

Voor de aansturing van deze module wordt de (enige) seriële (UART) verbinding van de micro:bit gebruikt.
Hierover worden "AT"-opdrachten verstuurd naar de module.
De baudrate van deze verbinding is 115200.
De gebruikte pins staan gedocumenteerd op het bordje: microbit Rx in pin12, Tx is pin8.

Een beschrijving van de beschikbare AT-opdrachten is te vinden in de documentatie: https://docs.espressif.com/projects/esp-at/en/release-v2.1.0.0_esp8266/AT_Command_Set/MQTT_AT_Commands.html#cmd-MQTTCONN

* https://www.elecfreaks.com/iot-bit-for-micro-bit.html
* https://www.elecfreaks.com/learn-en/microbitKit/iot_kit/iot_bit.html (wiki)

> Omdat er maar één UART verbinding tegelijk mogelijk is, kan de communicatie met de ESP8266 niet gecombineerd worden met de verbinding met de host. Dit betekent dat bijv. print-opdrachten dan niet mogelijk zijn (debugging!). Dit betekent dat we naar andere vormen van debugging moeten zoeken, bijv. via het display?
>
> Dat is waarschijnlijk ook wel aantrekkelijk om de status van de gateway, bijvoorbeeld bij het opstarten, te kunnen volgen.

Bij de AT-opdrachten zijn de volgende zaken van belang:

* je moet een bepaalde periode wachten voordat je de volgende opdracht geeft.
    * dat wordt in de onderstaande code uitgewerkt als "basic.pause(time)" - maar uiteindelijk willen we daarvoor liever een oplossing met timers gebruiken, in plaats van een dergelijke blokkerende actie.
* de AT-opdrachten komen gewoonlijk met een response, als reactie op de AT-opdracht. In principe kun je pas de volgende opdracht geven na ontvangst van deze response. (Je hebt dan geen timer nodig???)
* soms komt er (tussendoor) een onverwacht (asynchroon) bericht van de module, bijvoorbeeld bij de ontvangst van een MQTT-bericht. Dit is dan niet op basis van een direct voorafgaande AT-opdracht.
    * hoe herken je een dergelijk asynchroon bericht?


In [4]:
def sendAT(command: str, wait: int = 0):
    uart.write(f'{command}\r\n')
    sleep(wait) # basic.pause(wait)

(Gebruik van Python templates: `f'..string...{var}...'` - is dat een var, of zelfs een expr?).

(Hierboven, gebruik van `microbit.sleep(ms)`; alternatief is het gebruik van `utime.sleep_ms`. Wat is het verschil?)

In [2]:
def resetEsp8266():
    sendAT('AT+RESTORE', 1000) # restore to factory settings
    sendAT('AT+RST', 1000)     # reset, restart
    serial.readString()
    sendAT('AT+CWMODE=1', 500) # set to STA mode
    sendAT('AT+SYSTIMESTAMP=1634953609130', 100) # Set local timestamp.
    sendAT('AT+CIPSNTPCFG=1,8,"ntp1.aliyun.com","0.pool.ntp.org","time.google.com"', 100)

Uit de Espressif documentatie (van een oudere versie...)

Notes:

1. Not all AT Command has four commands.
2. [] = default value, not required or may not appear`
3. String values require double quotation marks, for example: `AT+CWSAP="ESP756290", "21030826", 1, 4`
4. Baudrate = 115200
5. AT Commands has to be capitalized, and end with "/r/n"

```Python
def init_ESP8266_connection():
    set serial pins: Tx=pin8 , Rx=pin12
    set baudrate 115200
```

microbit.uart.init(baudrate=115200,tx=pin8,rx=pin12)

**Opmerking:** `microbit.uart.init(baudrate=115200)` herstelt de verbinding met de host (via USB).

## Main loop

Er kunnen berichten komen van verschillende kanten (en op willekeurige momenten)

* een radio-bericht van een microbit in het netwerk
* een MQTT-bericht vanuit het internet
    * via "subscribe"


```Python
from microbit import *
import uart


while True:
    if uart.any():
        in_bytes = uart.read()
        in_buffer += str(in_bytes)
        # may contain CRLF?
    msg = radio.receive_bytes()
    if msg != None:
        pass
        # message from microbit in local network
```

## Gebruik van timers? Alternatief?

Hoe kunnen we op een andere manier omgaan met de interactie tussen de microbit en de ESP8266, zodat er geen "sleep" meer nodig is?

* in principe geeft een opdracht altijd een response, ten teken dat deze uitgevoerd is, al dan niet met fouten.
* je kunt wachten op deze response, voordat je een volgende opdracht geeft. Maar hoe programmeer je dat netjes?
    * vgl. de "promisse" aanpak? (Continuation meegeven als functie?)
    * gebruik van asyncio? (https://docs.micropython.org/en/latest/library/uasyncio.html)
    * is deze beschikbaar voor de microbit? Staat niet beschreven in de microbit micropython documentatie.

## on data received functie

```
serial.onDataReceived(serial.delimiters(Delimiters.NewLine), function() {})
```

* https://github.com/elecfreaks/pxt-esp8266iot/blob/master/ESP8266.ts

zie r. 364. De betreffende function wordt aangeroepen als er seriële input beschikbaar is - met daarin de genoemde delimiters. Deze input wordt verwerkt door de betreffende functie (en uit de input buffer verwijderd?)

* er wordt eerst gecontroleerd op "onverwachte input", die niet samenhangt met een direct voorafgaande opdracht. - bijvoorbeeld, een binnenkomend MQTT bericht.
* als het geen onverwachte input is, wordt afhankelijk van de direct voorafgaande opdracht, de input verwerkt.
    * afhankelijk van de inhoud van de respons, kan de status van het systeem veranderen; bijvoorbeeld: wifi-verbinding OK, of MQTT-verbinding OK. Bepaalde acties zijn alleen mogelijk bij een bepaalde toestand.

## Debugging

Idee: gebruik de twee buttons van de microbit.

* button A: verbindt de microbit met de host, en schrijft de gebufferde debug-i/o weg
    * we hebben dan een buffer nodig voor deze i/o; of een buffer van wat anders de print-opdrachten zouden zijn, voor debugging.
* button B: verbindt de microbit met de ESP8266, en initialiseert de buffers en de communicatie.


```Python
from microbit import *
import uart

debug_buffer = ''
input_buffer = ''
last_cmd = ''

# form of command: AT+cmd,arg0,arg1,arg2,arg3

def sendAT(command: str, wait: int = 0):
    uart.write(f'{command}\r\n')
    sleep(wait) # basic.pause(wait)

def handle_inputline(line: str):
    pass

def init_ESP8266():
    sendAT('AT+RESTORE', 1000) # restore to factory settings
    sendAT('AT+RST', 1000)     # reset, restart
    serial.readString()
    sendAT('AT+CWMODE=1', 500) # set to STA mode    
    

while True:
    if button_a.was_pressed():
        uart.init(baudrate=115200) # connect to host
        print(debug_buffer)
        
    if button_b.was_pressed():
        uart.init(baudrate=115200, tx=pin8, rx=pin12) # connect to ESP8266
        debug_buffer = ''
        input_buffer = ''
        init_ESP8266
        
        
    if uart.any():
        in_bytes = uart.read()
        input_buffer += str(in_bytes)
        eol_pos = input_buffer.find('\n')
        if eol_pos > 0:
            line = input_buffer[0:eol_pos+1]        # get line and
            input_buffer = input_buffer[eol_pos+1:] # remove from input_buffer
            debug_buffer += '>>' + line
            handle_inputline(line)

```        
        

## Gebruik van radio voor logging, debugging

We kunnen op de iot:bit geen gebruik maken van de host-communicatie: er is geen verbinding via USB (deze wordt alleen gebruikt voor de voeding van het bordje?)

Je moet de microbit **programmeren via de USB-verbinding op het microbit-bordje**; deze mag niet tegelijk met de USB-verbinding op de iot-bit actief zijn. De suggestie is om de microbit bij het programmeren steeds uit de iot:bit te verwijderen.
(Je kunt de iot-bit ook uit zetten; dat lijkt mij handiger...)

Omdat er eigenlijk geen andere verbinding mogelijk is, willen we de radio gebruiken voor het versturen van debuggings-gegevens.

Code voor de **debug-microbit:**

In [ ]:
from microbit import *
import radio

radio.config(length=128)
radio.on()

print("Remote logger")

while True:
    msg = radio.receive()
    if msg != None:
        print(msg)

Code voor de test-microbit:

In [ ]:
from microbit import *
import radio

radio.config(length=128)
radio.on()
cnt = 0

while True:
    radio.send("Test {x}".format(x=cnt))
    cnt += 1
    sleep(5000)

Code voor de gateway-microbit:

In [ ]:
from microbit import *
import radio

input_buffer = ''
last_cmd = ''
wifi_connected = False

# form of command: AT+cmd,arg0,arg1,arg2,arg3

def sendAT(command: str, wait: int = 0):
    uart.write('{cmd}\r\n'.format(cmd=command))
    sleep(wait) # basic.pause(wait)

def handle_inputline(line: str):
    global wifi_connected
    radio.send('>>' + line + '<<')
    if line.startswith('WIFI GOT IP'):
        display.show('C')
        wifi_connected = True
    return

def init_ESP8266():
    sendAT('AT+RESTORE', 1000) # restore to factory settings
    sendAT('AT+RST', 1000)     # reset, restart
    buf = uart.read()
    if buf != None:
        radio.send(">"+str(buf)+"<")
    sendAT('AT+CWMODE=1', 500) # set to STA mode    

def init_WiFi():
    sendAT('AT+CWJAP="{ssid}","{pw}"'.format(ssid='xxx', pw='xxx'), 5000)
    radio.send("...connected??")
    sendAT('AT+CIPSTAMAC?', 1000)
    sendAT('AT+CIPSTA?', 1000)

radio.config(length=128)
radio.on()

while True:
        
    if button_a.was_pressed():
        radio.send("A pressed")
        uart.init(baudrate=115200, tx=pin8, rx=pin12) # connect to ESP8266
        sleep(100)
        input_buffer = ''
        init_ESP8266()
    
    if button_b.was_pressed():
        radio.send("B pressed")
        init_WiFi()
          
    if uart.any():
        in_bytes = uart.read()
        if in_bytes != None:
            input_buffer += in_bytes.decode('utf-8')
            # radio.send('->' + input_buffer + '<-\r\n')

    eol_pos = input_buffer.find('\n')
    if eol_pos > 0:
        line = input_buffer[0:eol_pos-1]        # get line and
        input_buffer = input_buffer[eol_pos+1:] # remove from input_buffer
        handle_inputline(line)

Waaruit bestaat de initialisatie?

* initialisatie van de ESP8266
* WiFi-verbinding (lukt niet altijd in 1 keer)
* MQTT verbinding (idem; kan tussentijds wegvallen - ook door wegvallen van WiFi?)
    * eigenlijk moeten we in de hoofdlus controleren of deze verbinding er nog is, eventueel met een time-out
    * pas als deze er niet is hoeven we op WiFi te controleren.
* MQTT subscriptie
    * deze is steeds nodig als de verbinding met de broker (opnieuw) gemaakt is.

Maken van een MQTT-verbinding: (onderstaande is de Elecfreaks-JS-code)

```js
 export function connectMQTT(host: string, port: number, reconnect: boolean): void {
        mqtthost_def = host
        const rec = reconnect ? 0 : 1
        currentCmd = Cmd.ConnectMqtt
        sendAT(`AT+MQTTCONN=0,"${host}",${port},${rec}`)
        control.waitForEvent(EspEventSource, EspEventValue.ConnectMqtt)
        Object.keys(mqttSubscribeQos).forEach(topic => {
            const qos = mqttSubscribeQos[topic]
            sendAT(`AT+MQTTSUB=0,"${topic}",${qos}`, 1000)
        })
    }
```

in Python:

In [7]:
def connectMQTT(host: str, port: int, reconnect: bool):
    global mqtt_host
    mqtt_host = host
    current.cmd = "connectMQTT"
    sendAT('AT+MQTTCONN=0,"{hostnm}",{portnr},{rec}'.format(
        hostnm = host, 
        portnr = str(port), 
        rec = 1 if reconnect else 0
    ))
    ...wait for event: connectMQTT...
    ...subscribe to topics...

SyntaxError: invalid syntax (1062162232.py, line 10)

aoh(We hebben een lijst nodig van subscription-topics; die moeten we opgeven aan de broker. En bij een binnenkomend bericht moeten we nagaan of dat voor deze gateway bestemd is.)

Hoe implementeren we in dit geval van de "wait for event"? - dit betekent eigenlijk dat er gewacht wordt op een bepaald bericht van de ESP8266.

Kunnen er meerdere "events" tegelijk onderweg zijn? of worden die netjes sequentieel afgehandeld?

* *in dit geval* heeft het geen zin om nieuwe opdrachten te sturen, zolang de MQTT-verbinding nog niet actief is.
* we moeten die analyse voor alle gevallen maken?

```js
    /**
     * Check if ESP8266 successfully connected to mqtt broker
     */
    //% block="MQTT broker is connected"
    //% subcategory="MQTT" weight=24
    export function isMqttBrokerConnected() {
        return mqttBrokerConnected
    }

    /**
     * send message
     */
    //% subcategory=MQTT weight=21
    //% blockId=sendMQTT block="publish %msg to Topic:%topic with Qos:%qos"
    //% msg.defl=hello
    //% topic.defl=topic/1
    export function publishMqttMessage(msg: string, topic: string, qos: QosList): void {
        sendAT(`AT+MQTTPUB=0,"${topic}","${msg}",${qos},0`, 1000)
    }

    /**
     * disconnect MQTT broker
     */
    //% subcategory=MQTT weight=15
    //% blockId=breakMQTT block="Disconnect from broker"
    export function breakMQTT(): void {
        sendAT("AT+MQTTCLEAN=0", 1000)
    }

    //% block="when Topic: %topic have new $message with Qos: %qos"
    //% subcategory=MQTT weight=10
    //% draggableParameters
    //% topic.defl=topic/1
    export function MqttEvent(topic: string, qos: QosList, handler: (message: string) => void) {
        mqttSubscribeHandlers[topic] = handler
        mqttSubscribeQos[topic] = qos
    }
```

NB: voor de MQTT-verbinding moeten ook de username en password gezet worden (en het "schema" - via TCP, TLS, enz.?) Dit moet gebeuren voor de MQTTCONN.

Er zijn afzonderlijke opdrachten voor het zetten van de usernamen en password.

Hoe kunnen we nagaan of de MQTT-verbinding nog bestaat? Moeten we dat actief opvragen, of krijgen we daarvan automatisch bericht? (Idem, voor de WiFi-verbinding.)

Zie de MQTT notes achteraan:

> When the MQTT connection ends, it will prompt message `+MQTTDISCONNECTED:<LinkID>`
>
> When the MQTT connection established, it will prompt message `+MQTTCONNECTED:<LinkID>,<scheme>,<"host">,port,<"path">,<reconnect>`

```python
current = {"cmd": '', 'echo': False, 'complete': False, 'error': 0}
ok_state = False
wifi_connected = False
mqtt_connected = False

def on_wifi_connected():
    sendAT('AT+CIPSTAMAC?', 1000)
    sendAT('AT+CIPSTA?', 1000)

def on_mqtt_connected():
    pass

def set_current(cmd: str):
    current["cmd"] = cmd
    current["echo"] = False
    current["complete"] = False
    current["error"] = 0

def handle_line(line: str)
    global ok_state, current

    # first, handle unsollicited messages

    if line.startswith('MQTTSUBRECV'):
        display.show("S")
        return
    if line.startswith('WIFI CONNECTED'):
        wifi_connected = True
        display.show('B')
        return
    if line.startswith('WIFI GOT IP'):
        display.show('C')
        return

    # next,  handle complex (or all?) commands

    if current["cmd"] == 'CWJAP':
        if line.startswith('AT+CWJAP='):
            current["echo"] = True
            return
        if line.startswith('ERROR'):
            current["complete"] = True
            ok_state = False
            return
        if line.startswith('OK'):
            current["complete"] = True
            ok_state = True
            if wifi_connected:
                on_wifi_connected()
            return
        # else:
        log("unexpected-1: " + line)
    
    elif current.cmd == 'MQTTCONN':
        if line.startswith('AT+MQTTCONN='):
            current.echo = True
            return
        if line.startswith('OK'):
            mqtt_connected = True
            display.show('M')
            current["complete"] = True
            return
        if line.startswith('ERROR'):
            current.complete = True
            ok_state = False
            mqtt_connected = True
            on_mqtt_connected()
            return
        log("unexpected-2: " + line)
        return
    
    else:
        if line == '\r\n':
            return
        if line == 'OK\r\n':
            ok_state = True
            return
    log("unexpected-3: " + line)
    return     

```

## Voorbeelden

* zie https://docs.espressif.com/projects/esp-at/en/release-v2.1.0.0_esp8266/AT_Command_Examples/MQTT_AT_Examples.html

NB: er zijn veel opdrachten die alleen OK als resultaat hebben, of een foutcode. Daarvoor kunnen we waarschijnlijk een algemeen stramien ontwerpen.

### Voorbereidden van de MQTT connect

Voordat we de Connect opdracht kunnen geven, moeten we eerst de configuratie instellen.

* 

```
AT+MQTTUSERCFG=<LinkID>,<scheme>,<"client_id">,<"username">,<"password">,<cert_key_ID>,<CA_ID>,<"path">
```

* linkID = 0
* scheme = 1 (MQTT over TCP)
* client_id = (een unieke string - gebaseerd op MAC adres?)
* username = "mqtttest"
* password = "testmqtt"
* cert = , (of weglaten...)
* CA_ID = ,
* path = ,

> The total length of the entire AT command should be less than 256Bytes.

NB: als er meerdere clients zijn die zich aanmelden met dezelfde client_id, dan blijft alleen de laatste actief. De verbinding met de anderen wordt dan verbroken. Je moet dus wel zorgen dat die id uniek is, maar de waarde zelf doet er verder niet zoveel toe.

Je kunt het MAC-adres opvragen met: https://docs.espressif.com/projects/esp-at/en/release-v2.1.0.0_esp8266/AT_Command_Set/Wi-Fi_AT_Commands.html#cmd-STAMAC

```
AT+CIPSTAMAC?
Function: to obtain the MAC address of the ESP32 Station.
Response:
+CIPSTAMAC:<mac>
OK
```

Je kunt het IP-adres opvragen met: 

```
AT+CIPSTA?
Function: to obtain the IP address of the ESP32 Station.
Notice: Only when the ESP32 Station is connected to an AP can its IP address be queried.
Response:
+CIPSTA:<ip>
OK
```

## AT messages

De *messages* zijn unsollicited?

* https://docs.espressif.com/projects/esp-at/en/release-v2.1.0.0_esp8266/AT_Command_Set/index.html

In [8]:
test = {"aap": 12, "noot": "pinda", "mies": False}

In [10]:
test["aap"]

12

In [ ]:
from microbit import *
import radio

input_buffer = ''
last_cmd = ''
mac_address = ''

current = {"cmd": '', 'echo': False, 'complete': False, 'error': 0}

def set_current(cmd: str):
    global current
    current["cmd"] = cmd
    current["echo"] = False
    current["complete"] = False
    current["error"] = 0

def log(txt: str):
    radio.send(txt)
    
# form of command: AT+cmd=arg0,arg1,arg2,arg3

def sendAT(command: str, wait: int = 0):
    cmd = command[3:] # remove AT+ part
    pos = cmd.find('=')
    if pos >= 0:
        cmd = cmd[0:pos] # remove = part
    else:
        pos = cmd.find('?')
        if pos >= 0:
            cmd = cmd[0:pos] # remove ? part
    log(".." + cmd)        
    set_current(cmd)  
    uart.write('{cmd}\r\n'.format(cmd=command))
    sleep(wait) # basic.pause(wait)

ok_state = False
wifi_connected = False
mqtt_connected = False
ip_address = ""
mac_address = ""
netmask = ""

def on_wifi_connected():
    sendAT('AT+CIPSTAMAC?', 1000)

def on_mqtt_connected():
    pass

def on_mac_address_complete():
    sendAT('AT+CIPSTA?', 1000)
    
def on_ip_address_complete():
    log("ip address complete")

def handle_inputline(line: str):
    global ok_state, current, wifi_connected, mqtt_connected, mac_address, ip_address
    radio.send('>>' + line + '<<')

    # first, handle unsollicited messages

    if line.startswith('MQTTSUBRECV'):
        display.show("S")
        return
    if line.startswith('WIFI CONNECTED'):
        wifi_connected = True
        display.show('B')
        return
    if line.startswith('WIFI GOT IP'):
        display.show('C')
        return

    # next,  handle complex (or all?) commands

    if current["cmd"] == 'CWJAP':
        if line.startswith('AT+CWJAP='):
            current["echo"] = True
            return
        if line.startswith('+CWJAP:'):
            arg = line[7:]
            current["error"] = int(arg)
            display.show(arg)
            return
        if line.startswith('ERROR'):
            current["complete"] = True
            ok_state = False
            return
        if line == 'OK':
            current["complete"] = True
            display.show('D')
            ok_state = True
            if wifi_connected:
                on_wifi_connected()
            return
        log("unexpected-1: " + line)
        
    elif current["cmd"] == "CIPSTAMAC":
        if line.startswith("AT+CIPSTAMAC"):
            current["echo"] = True
            return
        if line.startswith("+CIPSTAMAC:"):
            arg = line[11:]
            mac_address = arg.replace(':', '')
            log("mac-addr:" + mac_address)
            return
        if line == 'OK':
            on_mac_address_complete()
            return
        
    elif current["cmd"] == "CIPSTA":
        if line.startswith("AT+CIPSTA"):
            current["echo"] = True
            return
        if line.startswith("+CIPSTA:ip:"):
            ip_address = line[11:]
            log("ip-addr:" + ip_address)
            return
        if line.startswith("+CIPSTA:gateway:"):
            gateway_address = line[16:]
            log("gateway-addr:" + gateway_address)
            return
        if line.startswith("+CIPSTA:netmask:"):
            netmask = line[16:]
            log("netmask:" + netmask)
            return         
        if line == 'OK':
            on_ip_address_complete()
            return    
    
    elif current["cmd"] == 'MQTTCONN':
        if line.startswith('AT+MQTTCONN='):
            current["echo"] = True
            return
        log("unexpected-2: " + line)
        return
    
    else:
        if line.startswith('AT+' + current["cmd"]):
            current["echo"] = True
    if line == 'OK':
        current["complete"] = True
        ok_state = True
        return
    if line == 'ERROR':
        current["complete"] = True
        ok_state = False
        return
    
    log("unexpected-3: " + line)
    return

def init_ESP8266():
    sendAT('AT+RESTORE', 1000) # restore to factory settings
    sendAT('AT+RST', 1000)     # reset, restart
    buf = uart.read()
    if buf != None:
        radio.send(">"+str(buf)+"<")
    sendAT('AT+CWMODE=1', 500) # set to STA mode    

def init_WiFi():
    sendAT('AT+CWJAP="{ssid}","{pw}"'.format(ssid='xxx', pw='xxx'), 5000)
    radio.send("...connected??")

radio.config(length=128)
radio.on()

while True:
        
    if button_a.was_pressed():
        radio.send("A pressed")
        uart.init(baudrate=115200, tx=pin8, rx=pin12) # connect to ESP8266
        sleep(100)
        input_buffer = ''
        init_ESP8266()
    
    if button_b.was_pressed():
        radio.send("B pressed")
        init_WiFi()
          
    if uart.any():
        in_bytes = uart.read()
        if in_bytes != None:
            input_buffer += in_bytes.decode('utf-8')
            # radio.send('->' + input_buffer + '<-\r\n')

    eol_pos = input_buffer.find('\n')
    if eol_pos > 0:
        line = input_buffer[0:eol_pos-1]        # get line and
        input_buffer = input_buffer[eol_pos+1:] # remove from input_buffer
        if line != '':
            handle_inputline(line)

Wachten tot de OK of ERROR response, ten teken dat een opdracht afgerond is, en de ESP8266 klaar is voor de volgende opdracht:

```python
def check_at():
    global input_buffer
    
    if uart.any():
        in_bytes = uart.read()
        if in_bytes != None:
            input_buffer += in_bytes.decode('utf-8')
            # radio.send('->' + input_buffer + '<-\r\n')

    eol_pos = input_buffer.find('\n')
    if eol_pos > 0:
        line = input_buffer[0:eol_pos-1]        # get line and
        input_buffer = input_buffer[eol_pos+1:] # remove from input_buffer
        if line != '':
            handle_inputline(line)
            
def wait_at_completed():
    while not (at_ok or at_error):
        check_wifi()
```

We hebben nu o.a. de variabelen `ok_state` en `error_state`, hernoemen tot `at_ok` en `at_error`.

Volgt er in het geval van een query ook een OK? - Ja, zo te zien aan de log van zowel CIPSTA als CIPSTAMAC.


NB: eigenlijk moeten we niet alleen de naam, maar ook de `=` of `?` meenemen als onderdeel van het AT-commando, bij de verwerking in de `handle_inputline`. (Eigenlijk: `at_handle_line`? of `handle_at_line`?)
Immers, de response die we verwachten hangt ook af van het type commando.

De OK en ERROR exits zijn voor veel opdrachten gemeenschappelijk. Het verschil is dat in sommige gevallen bij OK een speciale on-functie aangeroepen wordt.

**Exceptions.** Eén van de manieren om fouten te signaleren in Python, is het gebruik van exceptions. Levert dat in onze voorbeelden voordelen op?

Welke functies willen we definiëren - in de context van de gateway, maar mogelijk ook andere Python netwerk-toepassingen?

* initialiseren van de module (reset)
* opzetten van de WiFi-verbinding
* opvragen van het MAC-adres
* opvragen van het IP-adres (heeft alleen zin als er een IP-verbinding is)
* opzetten van de MQTT-verbinding
* abonneren op een MQTT topic

**Opmerking**. Tijdens het opzetten van de verbindingen van de gateway doen de andere inputs er niet toe: je kunt de invoer daarvan toch niet verwerken.


**NB** De wachttijd kan in het geval van expliciet wachten op OK of ERROR volgens mij gewoon 0 zijn. Maar daar moeten we dan wel mee experimenteren.

In [ ]:
def at_wifi_connect(ssid: str, pwd: str):
    sendAT('AT+CWJAP="{ssid}","{pw}"'.format(ssid='xxx', pw='xxx'), 5000)
    wait_at+completed()
    if at_ok:
        if wifi_connected:
            on_wifi_connected()
        sendAT('AT+CIPSTAMAC?', 1000)
        wait_at_completed()
        sendAT('AT+CIPSTA?', 1000)
        wait_at_completed()     

**Foutsignalering** Bijvoorbeeld bij het opbouwen van een WiFi-verbinding: CWJAP.

```
>>+CWJAP:4<<
>><<
>>ERROR<<
```

Je krijgt dan eerst de foutcode, en daarna "ERROR". De invoer `+CWJAP:xx` komt alleen in het geval van een fout, niet bij correcte verwerking. (4 staat voor: connection failed; daar word je natuurlijk veel wijzer van...)

Hoe worden de verschillende vormen genoemd?

* Test (`=?`) - *voorbeeld???*
* Query (`?`) - 
* Set (`=`)
* Execute (``)

Eigenlijk horen die tekens bij het commando - omdat de response ook afhangt van de vorm.

NB: ik begrijp dat ERROR ook unsollicited kan voorkomen.

In [ ]:
def at_mqtt_set_userconfig(scheme: int, client_id: str, username: str, password; str):
    sendAT('AT+MQTTUSERCFG=1,{a},"{b}","{c}","{d}"'.format(
        a = scheme,
        b = client_id,
        c = username,
        d = password
    ), 1000)
    wait_at_completed()

Wij gebruiken voorlopig alleen scheme: 1, mqtt over TCP

In [ ]:
def at_mqtt_connect(host, port, reconnect):
    sendAT('AT+MQTTCONN=1,"{a}","{b}",{c}'.format(
        a = host,
        b = port,
        c = reconnect
    ), 1000)
    wait_at_completed()

Wat zijn de AT-opdrachten waarmee er een complexere interactie is?

* de queries: hierbij wordt er altijd een response gegeven. (De vorm daarvan is overigens goed af te leiden van de opdracht.)

Ik heb in het voorgaande een speciale vorm voor MQTTCONN opgenomen, maar volgens mij is dat eigenlijk niet nodig: deze valt onder de generieke vorm van OK (en ERROR?).

* wordt er in het geval van een fout, mislukte verbinding, ook een foutcode gegeven?
* hierover staat niets gedocumenteerd, maar ik kan mij dat niet voorstellen...
* ik zou ongeveer dezelfde foutcodes verwachten als in het geval van de WiFi connectie.

Hieronder volgt een volgende versie van de gateway code, nu met "wait_at_completed".

In [ ]:
from microbit import *
import radio

wifi_ssid = 'xxx'
wifi_password = 'xxx'

last_cmd = ''

current = {"cmd": '', 'echo': False}
at_ok = False
at_error = False
at_errorcode = 0

def set_current(cmd: str):
    global current
    current["cmd"] = cmd
    current["echo"] = False

def log(txt: str):
    radio.send(txt)
    
# form of command: AT+cmd=arg0,arg1,arg2,arg3

def sendAT(command: str, wait: int = 0):
    global at_ok, at_error, at_errorcode
    at_ok = False
    at_error = False
    at_errorcode = 0
    
    cmd = command[3:] # remove AT+ part
    pos = cmd.find('=')
    if pos >= 0:
        cmd = cmd[0:pos] # remove = part
    else:
        pos = cmd.find('?')
        if pos >= 0:
            cmd = cmd[0:pos] # remove ? part
    log(".." + cmd)        
    set_current(cmd)  
    uart.write('{cmd}\r\n'.format(cmd=command))
    sleep(wait) # basic.pause(wait)

wifi_connected = False
mqtt_connected = False
ip_address = ""
mac_address = ""
netmask = ""

def on_wifi_connected():
    log("on wifi connected")

def on_mqtt_connected():
    log("on mqtt connected")

def handle_inputline(line: str):
    global at_ok, at_error, at_errorcode, current, mac_address, ip_address, gateway_address, netmask
    log('>>' + line + '<<')

    # first, handle unsollicited messages

    if line.startswith('MQTTSUBRECV'):
        display.show("S")
        return
    if line.startswith('WIFI CONNECTED'):
        wifi_connected = True
        display.show('B')
        return
    if line.startswith('WIFI GOT IP'):
        display.show('P')
        return

    # next,  handle complex (or all?) commands

    if current["cmd"] == 'CWJAP':
        if line.startswith('+CWJAP:'):
            arg = line[7:]
            at_errorcode = int(arg)
            display.show(arg)
            return
        
    elif current["cmd"] == "CIPSTAMAC":
        if line.startswith("+CIPSTAMAC:"):
            arg = line[11:]
            mac_address = arg.replace(':', '')
            log("mac-addr:" + mac_address)
            return
        
    elif current["cmd"] == "CIPSTA":
        if line.startswith("+CIPSTA:ip:"):
            ip_address = line[11:]
            log("ip-addr:" + ip_address)
            return
        if line.startswith("+CIPSTA:gateway:"):
            gateway_address = line[16:]
            log("gateway-addr:" + gateway_address)
            return
        if line.startswith("+CIPSTA:netmask:"):
            netmask = line[16:]
            log("netmask:" + netmask)
            return    
    else:
        pass
  
    if line.startswith('AT+' + current["cmd"]):
        current["echo"] = True
        return
    if line == 'OK':
        at_ok = True
        return
    if line == 'ERROR':
        at_error = True
        return
    
    log("unexpected-3: " + line)
    return

def init_ESP8266():
    sendAT('AT+RESTORE', 1000) # restore to factory settings
    sendAT('AT+RST', 1000)     # reset, restart
    buf = uart.read()
    if buf != None:
        log(">"+str(buf)+"<")
    sendAT('AT+CWMODE=1', 500) # set to STA mode
    wait_at_completed()
    display.show('A')
    
# the delay for sendAT is set to 0: wait_at_completed takes case of timing
    
def at_wifi_connect(ssid: str, password: str):
    sendAT('AT+CWJAP="{a}","{b}"'.format(a=ssid, b=password), 0)
    wait_at_completed()
    if at_ok:
        if wifi_connected:
            on_wifi_connected()
            display.show('C')
        sendAT('AT+CIPSTAMAC?', 0)
        wait_at_completed()
        sendAT('AT+CIPSTA?', 0)
        wait_at_completed()    

radio.config(length=128)
radio.on()

input_buffer = ''

def at_check_input():
    global input_buffer
    
    if uart.any():
        in_bytes = uart.read()
        if in_bytes != None:
            input_buffer += in_bytes.decode('utf-8')

    eol_pos = input_buffer.find('\n')
    if eol_pos > 0:
        line = input_buffer[0:eol_pos-1]        # get line and
        input_buffer = input_buffer[eol_pos+1:] # remove from input_buffer
        if line != '':
            handle_inputline(line)
    
def wait_at_completed():
    while not (at_ok or at_error):
        at_check_input()

while True:      
    if button_a.was_pressed():
        log("A pressed")
        uart.init(baudrate=115200, tx=pin8, rx=pin12) # connect to ESP8266
        sleep(100)
        input_buffer = ''
        init_ESP8266()
    
    if button_b.was_pressed():
        log("B pressed")
        at_wifi_connect(wifi_ssid, wifi_password)
          
    at_check_input()

* het heeft eigenlijk geen zin om de echo-gegevens bij te houden. Eventueel kunnen we een globale boolean "at_echo" bijhouden?
* de manier waarop we de huidige opdracht vastleggen kunnen we verbeteren:
    * *inclusief* de `=` of `?`.
    * expliciet meegeven als afzonderlijke onderdelen, in plaats van decoderen uit de complete opdracht?
* de knoppen kunnen we voor verschillende functies gebruiken, afhankelijk van de toestand
    * knop B: als er nog geen wifi verbinding is: maak wifi verbinding
    * knop B: als er al een wifi verbinding is (en geen mqtt): maak mqtt verbinding
    * knop B: als er een mqtt verbinding is: stuur mqtt bericht.
    * elk van deze deelopdrachten eindigt met `wait_at_completed()`.

In de bovenstaande code maak ik nogal eens gebruik van de (constance) positie van een scheidingsteken, zoals een `:`. We kunnen beter een functie maken die de rest van de string na een scheidingsteken oplevert.

## MQTT

* configureren van MQTT gebruiker (username, password)
* verbinding maken met MQTT
* publiceren van een bericht via MQTT
* abonneren op MQTT topics
* ontvangen van MQTT berichten



NB: miscchien kunnen we de client-id hierin weglaten, en alvast een client-id afgeleid van het MAC-adres invullen. (Een client-id is max. 23 bytes lang. Het MAC-adres is 12 (hex) tekens lang. Bijvoorbeeld: "gw-" gevolgd door het mac-adres?)

Het schema is (voorlopig) alleen 1: MQTT over TCP.

In [ ]:
def at_mqtt_set_userconfig(scheme: int, client_id: str, username: str, password; str):
    sendAT('AT+MQTTUSERCFG=0,{a},"{b}","{c}","{d}"'.format(
        a = scheme,
        b = client_id,
        c = username,
        d = password
    ), 0)
    wait_at_completed()

In [ ]:
def at_mqtt_connect(host, port, reconnect):
    sendAT('AT+MQTTCONN=0,"{a}","{b}",{c}'.format(
        a = host,
        b = port,
        c = reconnect
    ), 0)
    wait_at_completed()

In [ ]:
from microbit import *
import radio

wifi_ssid = 'xxx'
wifi_password = 'xxx'
mqtt_user = 'mqtttest'
mqtt_password = 'testmqtt'
mqtt_host = 'infvopedia.nl'
mqtt_port = 1883
mqtt_tcp = 1

last_cmd = ''

current = {"cmd": '', 'echo': False}
at_ok = False
at_error = False
at_errorcode = 0

def set_current(cmd: str):
    global current
    current["cmd"] = cmd
    current["echo"] = False

def log(txt: str):
    radio.send(txt)
    
# form of command: AT+cmd=arg0,arg1,arg2,arg3

def sendAT(command: str, wait: int = 0):
    global at_ok, at_error, at_errorcode
    at_ok = False
    at_error = False
    at_errorcode = 0
    
    cmd = command[3:] # remove AT+ part
    pos = cmd.find('=')
    if pos >= 0:
        cmd = cmd[0:pos] # remove = part
    else:
        pos = cmd.find('?')
        if pos >= 0:
            cmd = cmd[0:pos] # remove ? part
    log(".." + cmd)        
    set_current(cmd)  
    uart.write('{a}\r\n'.format(a=command))
    sleep(wait) # basic.pause(wait)

wifi_connected = False
mqtt_connected = False
ip_address = ""
mac_address = ""
netmask = ""

def on_wifi_connected():
    log("on wifi connected")

def on_mqtt_connected():
    log("on mqtt connected")

def handle_inputline(line: str):
    global at_ok, at_error, at_errorcode, current, wifi_connected, mac_address, ip_address, gateway_address, netmask
    log('>>' + line + '<<')

    # first, handle unsollicited messages

    if line.startswith('WIFI CONNECTED'):
        wifi_connected = True
        display.show('B')
        return
    if line.startswith('WIFI GOT IP'):
        display.show('P')
        return

    # next,  handle complex (or all?) commands

    if current["cmd"] == 'CWJAP':
        if line.startswith('+CWJAP:'):
            arg = line[7:]
            at_errorcode = int(arg)
            display.show(arg)
            return
        
    elif current["cmd"] == "CIPSTAMAC":
        if line.startswith("+CIPSTAMAC:"):
            arg = line[11:]
            mac_address = arg.replace(':', '').replace('"','')
            log("mac-addr:" + mac_address)
            return
        
    elif current["cmd"] == "CIPSTA":
        if line.startswith("+CIPSTA:ip:"):
            ip_address = line[11:]
            log("ip-addr:" + ip_address)
            return
        if line.startswith("+CIPSTA:gateway:"):
            gateway_address = line[16:]
            log("gateway-addr:" + gateway_address)
            return
        if line.startswith("+CIPSTA:netmask:"):
            netmask = line[16:]
            log("netmask:" + netmask)
            return
        
    elif current["cmd"] == "MQTTCONN":
        if line.startswith("+MQTTCONNECTED:"):  # kan ook unsollicited...
            log("mqtt connected")
            mqtt_connected = True
            display.show('M')
            return
    else:
        pass
  
    if line.startswith('AT+' + current["cmd"]):
        current["echo"] = True
        return
    if line == 'OK':
        at_ok = True
        return
    if line == 'ERROR':
        at_error = True
        return
    
    log("unexpected-3: " + line)
    return

def init_ESP8266():
    global wifi_connected, mqtt_connected
    wifi_connected = False
    mqtt_connected = False
    sendAT('AT+RESTORE', 1000) # restore to factory settings
    sendAT('AT+RST', 1000)     # reset, restart
    buf = uart.read()
    if buf != None:
        log(">"+str(buf)+"<")
    sendAT('AT+CWMODE=1', 500) # set to STA mode
    wait_at_completed()
    display.show('A')
    
# the delay for sendAT is set to 0: wait_at_completed takes care of timing
    
def at_wifi_connect(ssid: str, password: str):
    sendAT('AT+CWJAP="{a}","{b}"'.format(a=ssid, b=password), 0)
    wait_at_completed()
    if at_ok:
        if wifi_connected:
            on_wifi_connected()
            display.show('C')
        sendAT('AT+CIPSTAMAC?', 0)
        wait_at_completed()
        sendAT('AT+CIPSTA?', 0)
        wait_at_completed()
        
def at_mqtt_set_userconfig(scheme: int, client_id: str, username: str, password: str):
    sendAT('AT+MQTTUSERCFG=0,{a},"{b}","{c}","{d}",0,0,""'.format(
        a = scheme,
        b = client_id,
        c = username,
        d = password
    ), 0)
    wait_at_completed()
    
def at_mqtt_connect(host, port, reconnect):
    sendAT('AT+MQTTCONN=0,"{a}",{b},{c}'.format(
        a = host,
        b = port,
        c = reconnect
    ), 0)
    wait_at_completed()    
        
def at_mqtt_start():
    global mqtt_connected
    
    at_mqtt_set_userconfig(1, 'gw-'+mac_address, mqtt_user, mqtt_password)
    at_mqtt_connect(mqtt_host, mqtt_port, 1)
    if at_ok:
        mqtt_connected = True
        display.show('M')
        on_mqtt_connected()

def at_mqtt_publish(topic: str, data: str, qos: int, retain: bool):
    sendAT('AT+MQTTPUB=0,"{a}","{b}",{c},{d}'.format(
        a = topic,
        b = data,
        c = qos,
        d = 1 if retain else 0
    ))
    
def at_mqtt_subscribe(topic: str, qos: int):
    sendAT('AT+MQTTSUB=0,"{a}",{b}'.format(
        a = topic,
        b = qos
    ))
    
def on_mqtt_message(topic: str, data: str):
    log('msg-topic: ' + topic)
    log(data)
        
radio.config(length=128)
radio.on()

input_buffer = ''

def at_check_input():
    global input_buffer
    
    if uart.any():
        in_bytes = uart.read()
        if in_bytes != None:
            input_buffer += in_bytes.decode('utf-8')
            
    if input_buffer.startswith('+MQTTSUBRECV:0'):
        parts = input_buffer.split(',')
        if len(parts) < 4: 
            return
        topic = parts[1].strip('"')
        datasize = int(parts[2])
        log("mqtt-datasize: " + parts[2])
        if len(parts[3]) < datasize + 2:  # incl. CRLF
            return
        totalsize = len(parts[0]) + len(parts[1]) + len(parts[2]) + datasize + 5
        # 3 comma's and CRLF
        input_buffer = input_buffer[totalsize:]
        on_mqtt_message(topic, parts[3][0:datasize])
        return
        
    eol_pos = input_buffer.find('\n')
    if eol_pos > 0:
        line = input_buffer[0:eol_pos-1]        # get line and
        input_buffer = input_buffer[eol_pos+1:] # remove from input_buffer
        if line != '':
            handle_inputline(line)
    
def wait_at_completed():
    while not (at_ok or at_error):
        at_check_input()

while True:      
    if button_a.was_pressed():
        log("A pressed")
        uart.init(baudrate=115200, tx=pin8, rx=pin12) # connect to ESP8266
        sleep(100)
        input_buffer = ''
        init_ESP8266()
    
    if button_b.was_pressed():
        log("B pressed")
        if not wifi_connected:
            at_wifi_connect(wifi_ssid, wifi_password)
        elif not mqtt_connected:
            at_mqtt_start()
        elif mqtt_connected:
            at_mqtt_publish("microbit/test", "Succes!", 0, False)
            wait_at_completed()
            at_mqtt_subscribe("microbit/test", 0)
            wait_at_completed()
          
    at_check_input()

Het bovenstaande werkt! (22-4-2023, 17:30). De berichten worden door de broker (en de andere clients) netjes ontvangen.

(De subscribe is 23 april toegevoegd, en werkte ook (bijna) direct goed: het afhandelen van de inkomende data die over meerdere regels verdeeld is, gaat nog niet goed.)

We weten dat de LFs niet doorgegeven worden; maar het teken daarvoor zou wel doorgegeven moeten worden (ook als het een CR is???) - we moeten mogelijk een andere strategie gebruiken.



De volgende stappen zijn:

* ontvangen van mqtt-berichten
* integreren met de gateway-code (zoals al eerder gemaakt, voor de ESP32 gateway).

De structuur van de ontvangen mqtt-berichten wijkt wat af van de normale structuur.

Mogelijke aanpak:

* bij de aanroep van `handle_input_line` geven we de regel inclusief de (CR)LF door. Deze maken mogelijk deel uit van de data, deze kunnen we (nog) niet verwijderen.
* in `handle_input_line` controleren we als eerste op een inkomend mqtt-bericht. Het kan zijn dat dit nog niet compleet is: we moeten dan rest van het bericht inlezen. (...eigenlijk kan dat niet met behulp van uart-opdrachten die niet het complete bericht opleveren; we willen geen blokkerende acties.)
    * om het goed te doen, moeten we (asynchroon) de uart-leesopdrachten geven totdat het bericht compleet is. Dat kan eigenlijk niet in handle_input-line, maar moet dan in de at_check_input-functie.


(27-april 17:00) Het (afzonderlijk) ontvangen van berichten **lijkt nu goed te werken**; de max. grootte die ontvangen wordt is 128? - (klopt de lengte dan wel? en, als de lengte niet klopt, wordt er dan ook teveel data uit de invoerbuffer gelezen?)

Met andere woorden: ik moet bij het verwerken van de invoerdata van een bericht rekening houden met deze lengte-problemen.

Het wordt ook tijd om van de test-structuur af te stappen, en een meer definitieve opzet van het programma te maken. 

Waar ik nog niet uit ben: gebruiken we daarvoor een aparte library? En moeten we die ook kunnen gebruiken voor de microbit als iot-device (in plaats van de implementatie als gateway)?

Er verschijnt zo nu en dan ook een bericht dat de mqtt-verbinding verbroken is. (En, omdat de instelling zo is dat die verbinding automatisch hersteld wordt, ook een bericht dat de verbinding weer werkt.)

(Kunnen hierdoor berichten verloren gaan? We moeten dan weer berichten gaan nummeren...)

## Organiseren als library

Kunnen we e.e.a. organiseren als library, die we kunnen gebruiken (i) voor een microbit als IoT-device; (ii) voor een microbit als IoT-gateway?

Sommige van deze functies zijn synchroon, d.w.z., pas als de opdracht succesvol uitgevoerd is, keert de functie terug.

Andere functies keren direct terug; maar er kan pas een volgende opdracht gegeven worden, als de vorige afgerond is.
Omdat het geen zin heeft lokale functies uit te voeren als er nog geen verbinding is (in elk geval voor een gateway), zijn sommige van deze functies synchroon: ze wachten tot ze klaar zijn.

* `at_busy` geeft aan dat er nog een functie bezig is uitgevoerd te worden. Er kan dan geen nieuwe functie uitgevoerd worden.
* de functie `at_check_input()` moet in de event-loop uitgevoerd worden. Als deze functie niet vaak genoeg aangeroepen wordt, loop je de kans om inputs van de module te missen.


Mogelijke functies:

* at_init (of reset) (+)
* at_wifi_connect (+)
* at_mqtt_configure (+)
* at_mqtt_connect (*)
* at_mqtt_is_connected (+)
* at_busy (of at_is_completed?)
* at_mqtt_subscribe
* at_mqtt_on_message (of subscribed message)
* at_mqtt_publish (*)
* at_wait_completed (+)
* at_check_input (**)

Er kan van alles misgaan bij het uitvoeren van een functie. Het resultaat is pas bekend als de functie uitgevoerd is. (Soms is er dan ook een foutcode beschikbaar.)

Een "asynchrone" functie kan in een synchrone omgezet worden door deze te laten volgen door:

`at_wait_completed()` (NB: deze functie is een vorm van "actief wachten": de input moet voortdurend gecontroleerd worden.)

(Heeft het dan wel zin om de synchrone varianten aan te bieden? De gebruiker kan die altijd zelf maken?)


(als je een functie aanroept terwijl de vorige nog actief is, wordt deze direct beëindigd?) Of: dan wordt gewacht tot de vorige functie klaar is - we draaien de synchronsatie dan eigenlijk om...

Een probleem is dat je de fouten dan mogelijk op de verkeerde plek signaleert, nl. bij de volgende functie.

We kunnen ook afspreken dat in het geval van een fout in de vorige functie, de volgende functie in een exception resulteert.
De meeste functies bij de initialisatie zijn afhankelijk van het succes van de voorafgaande functie(s).
Door het gebruik van exceptions hoeven we niet steeds op die mogelijke fouten te controleren.

Er zijn nu twee manieren om na te gaan of er een opdracht actief is:

* als zowel ok als error False zijn
* als de huidige opdracht niet leeg is.

**Gebruik van display: in de toepassing, niet in de library.** Voor de toepassing is het handig als je als gebruiker de status weet, bijvoorbeeld: verbonden met het WiFi netwerk, of verbonden met de MQTT broker.
In de context van de iot:bit heb je daarvoor eigenlijk alleen het display beschikbaar.

Het is niet handig om het display te gebruiken (claimen) vanuit een libraryfunctie. Dit moeten we dus verhuizen naar de toepassing.

## atesp library

(Ik vind de naam nog niet goed gekozen, deze is niet afhankelijk van ESP. Maar het is wel de ESP code en documentatie die we gebruiken als uitgangspunt.)

**Opmerkingen.**

* we slaan de volledige at-opdracht op, zonder de parameters, maar inclusief `AT+` en `=` of `?`. (Ik weet niet waarom niet...)

In [ ]:
from microbit import *
import radio

# global variables

at_ok = False
at_error = False
at_errorcode = 0
at_cmd = ''

wifi_connected = False
mqtt_connected = False
ip_address = ""
mac_address = ""
netmask = ""

# logging to remote microbit

def log(txt: str):
    radio.send(txt)

radio.config(length=128)
radio.on()

# low-level: send command to ESP module

def sendAT(command: str, wait: int = 0):
    uart.write('{a}\r\n'.format(a=command))
    sleep(wait)

def handle_inputline(line: str):
    global at_cmd, at_ok, at_error, at_errorcode, wifi_connected, mac_address, ip_address, gateway_address, netmask
    global mqtt_connected
    log('>>' + line + '<<')

    # first, handle unsollicited messages

    if line.startswith('WIFI CONNECTED'):
        wifi_connected = True
        display.show('B')
        return
    if line.startswith('WIFI GOT IP'):
        display.show('P')
        return
    if line.startswith("+MQTTCONNECTED:"):
        log("mqtt connected")
        mqtt_connected = True
        display.show('M')
        return
    if line.startswith("+MQTTDISCONNECTED:"):
        log("mqtt disconnected")
        mqtt_connected = False
        display.show('U')
        return 

    # next,  handle complex (or all?) commands

    if at_cmd == 'AT+CWJAP=':
        if line.startswith('+CWJAP:'):
            arg = line[7:]
            at_errorcode = int(arg)
            display.show(arg)
            return
        
    elif at_cmd == "AT+CIPSTAMAC?":
        if line.startswith("+CIPSTAMAC:"):
            arg = line[11:]
            mac_address = arg.replace(':', '').replace('"','')
            log("mac-addr:" + mac_address)
            return
        
    elif at_cmd == "AT+CIPSTA?":
        if line.startswith("+CIPSTA:ip:"):
            ip_address = line[11:]
            log("ip-addr:" + ip_address)
            return
        if line.startswith("+CIPSTA:gateway:"):
            gateway_address = line[16:]
            log("gateway-addr:" + gateway_address)
            return
        if line.startswith("+CIPSTA:netmask:"):
            netmask = line[16:]
            log("netmask:" + netmask)
            return
    else:
        pass
  
    if line.startswith(at_cmd):
        # echo
        return
    if line == 'OK':
        at_ok = True
        at_cmd = ''
        return
    if line == 'ERROR':
        at_error = True
        at_cmd = ''
        return
    
    log("unexpected-3: " + line)
    return

def log_mqtt_message(topic: str, msg: str):
    log('msg-topic: ' + topic)
    log(msg)
    
on_mqtt_message = log_mqtt_message

input_buffer = ''

def at_check_input():
    global input_buffer
    
    if uart.any():
        in_bytes = uart.read()
        if in_bytes != None:
            input_buffer += in_bytes.decode('utf-8')
            
    if input_buffer.startswith('+MQTTSUBRECV:0'):
        parts = input_buffer.split(',')
        if len(parts) < 4: 
            return
        topic = parts[1].strip('"')
        datasize = int(parts[2])
        log("mqtt-datasize: " + parts[2])
        if len(parts[3]) < datasize + 2:  # incl. CRLF
            return
        totalsize = len(parts[0]) + len(parts[1]) + len(parts[2]) + datasize + 5
        # 3 comma's and CRLF
        input_buffer = input_buffer[totalsize:]
        on_mqtt_message(topic, parts[3][0:datasize])
        return
        
    eol_pos = input_buffer.find('\n')
    if eol_pos > 0:
        line = input_buffer[0:eol_pos-1]        # get line and
        input_buffer = input_buffer[eol_pos+1:] # remove from input_buffer
        if line != '':
            handle_inputline(line)

# wait until last at-command is finished (OR or ERROR)
def wait_at_ready():
    global at_cmd
    while at_cmd != '': 
        at_check_input()
        
def at_is_ready()-> bool:
    global at_cmd
    return at_cmd == ''

# send AT command - only after prev. command has finished.
# ...may wait for several seconds, e.g. when connecting to WiFi, MQTT

def at_send(cmd: str):
    global at_ok, at_error, at_errorcode, at_cmd
    
    wait_at_ready()
    at_ok = False
    at_error = False
    at_errorcode = 0
    
    pos = cmd.find('=')
    if pos >= 0:
        at_cmd = cmd[0:pos+1] # remove parameters
    else:
        at_cmd = cmd
    log(".." + cmd)        
    sendAT(cmd, 0)

def at_init_ESP():
    global wifi_connected, mqtt_connected, input_buffer
    wifi_connected = False
    mqtt_connected = False
    input_buffer = ''
    
    sendAT('AT', 1000)         # wake-up
    sendAT('AT+RESTORE', 1000) # restore to factory settings
    sendAT('AT+RST', 1000)     # reset, restart
    buf = uart.read()
    if buf != None:
        log(">"+str(buf)+"<")
    at_send('AT+CWMODE=1') # set to STA mode
    wait_at_ready()
    display.show('A')

def at_wifi_connect(ssid: str, password: str):
    global at_errorcode
    
    at_send('AT+CWJAP="{a}","{b}"'.format(a=ssid, b=password))
    wait_at_ready()
    if at_ok and wifi_connected:
        return
    else:
        raise Exception("wifi connection failed", str(at_errorcode))

def at_wifi_connected() -> bool:
    global wifi_connected
    return wifi_connected

def at_get_mac_address() -> str:
    global mac_address
    at_send('AT+CIPSTAMAC?')
    wait_at_ready()
    return mac_address

def at_get_ip_address() -> str:
    global ip_adress, wifi_connected
    if not wifi_connected:
        return ""
    at_send('AT+CIPSTA?')
    wait_at_ready()
    return ip_address

def at_mqtt_set_userconfig(scheme: int, client_id: str, username: str, password: str):
    at_send('AT+MQTTUSERCFG=0,{a},"{b}","{c}","{d}",0,0,""'.format(
        a = scheme,
        b = client_id,
        c = username,
        d = password
    ))

def at_mqtt_connect(host, port, reconnect):
    at_send('AT+MQTTCONN=0,"{a}",{b},{c}'.format(
        a = host,
        b = port,
        c = reconnect
    ))

def at_mqtt_connected():
    global mqtt_connected
    return mqtt_connected

#def at_mqtt_start():
#    global mqtt_connected
#    
#    at_mqtt_set_userconfig(1, 'gw-'+mac_address, mqtt_user, mqtt_password)
#    at_mqtt_connect(mqtt_host, mqtt_port, 1)
#    if at_ok:
#        mqtt_connected = True
#        display.show('M')
#        on_mqtt_connected()

def at_mqtt_publish(topic: str, data: str, qos: int, retain: bool):
    at_send('AT+MQTTPUB=0,"{a}","{b}",{c},{d}'.format(
        a = topic,
        b = data,
        c = qos,
        d = 1 if retain else 0
    ))
    
def at_mqtt_subscribe(topic: str, qos: int):
    at_send('AT+MQTTSUB=0,"{a}",{b}'.format(
        a = topic,
        b = qos
    ))

# function (topic: str, msg: str)
def at_set_on_mqtt_message(function):
    global on_mqtt_message
    on_mqtt_message = function


Ik heb geen ervaring met exceptions in Python; en in het bijzonder niet met het meegeven van parameters aan exceptions. Eigenlijk moet dit wel getest worden!

Is de WiFi-verbinding nodig voor het opvragen van het MAC-adres, of kunnen we dat gewoon als een losse functie aanbieden? (Ik zou verwachten dat zodra je de WiFi-mode gezet hebt voor "station" (is eindpunt), het MAC-adres gedefinieerd is.)

Je moet in het geval van het opvragen van het mac-adres, wel wachten tot de at-functie klaar is: eerder is het resultaat niet bekend cq. gecommuniceerd. Je kunt deze functie aanroepen na het initialiseren van de ESP. (Of, eigenlijk, na het zetten van de "station" mode - daarvoor moeten we eigenlijk ook een aparte functie maken: `at_set_wifi_mode(int)`). Hierbij is mode=1 de "station mode"; mode=0 schakelt WiFi uit.)

Het MAC-adres hebben we nodig voor bijv. een unieke identificatie als IoT-gateway of IoT-knoop.

(Het IP-adres is in veel gevallen niet nodig. Ook daarvoor kunnen we beter een aparte functie definiëren.)

In principe geldt na de aanroep (en terugkeer) van at_wifi_connect dat er een wifi-verbinding is. Als het niet lukt om een verbinding te maken, dan moet deze functie een (i) foutresultaat opleveren; (ii) of, een exception raisen. (Wat een rotwoord... "signaleren" lijkt mij beter.)

(Het lijkt mij geen zin hebben om een asynchrone versie van wifi_connect aan te bieden: als er geen verbinding is, kun je toch niets met binnenkomende lokale berichten.)

* initialiseren van ESP-module
* initialieren van WiFi; verbinden met WiFi netwerk (ssid, passwd)
    * als de verbinding niet lukt: opnieuw proberen???
* configureren van mqtt; verbinden met mqtt broker (user, passwd)
* ...als mqtt verbinding wegvalt: opnieuw verbinden.

Wat moeten we doen als de WiFi verbinding niet lukt? Opnieuw proberen? (en als deze wegvalt?)

* nb: als je een lang-draaiende gateway of IoT-node hebt, dan kan het voorkomen dat de WiFi-verbinding wegvalt, bijvoorbeeld doordat het base-station opnieuw opgestart wordt. In dat geval moet het wegvallen van de verbinding gesignaleerd worden - en moet geprobeerd worden de verbinding opnieuw te maken, waarschijnlijk met een voldoend grote wachttijd.

Wat moeten we doen als de MQTT verbinding niet lukt? (en als de verbinding wegvalt?)

Wat moeten we in de loop doen?

Het onderstaande is een demonstratie van functie-variabelen in Python. Dit werkt zoals verwacht.

In [24]:
test = None

def demo (x):
    print("Hi there " + x)
    
demo('y')
test = demo
test('x')

Hi there y
Hi there x


---

In [ ]:
# AT-ESP gateway for microbit iot-network

from microbit import *
from espat import *
import radio

wifi_ssid = 'infvo-iot'
wifi_password = 'LISP77(Midas'
mqtt_user = 'mqtttest'
mqtt_password = 'testmqtt'
mqtt_host = 'infvopedia.nl'
mqtt_port = 1883

radio.on()

# logging is done by radio messages to the logging microbit
def log(msg: str):
    radio.send(msg)

# default handler for incoming mqtt messages
# should be re-defined by ????
def on_mqtt_message(topic: str, msg: str):
    log('mqtt-topic: ' + topic)
    log('mqtt-msg:' + msg)

# should this be done in the library??? (or here, in the gateway?)
uart.init(baudrate=115200, tx=pin8, rx=pin12) # connect to ESP8266
sleep(100)

display.show('0')
at_init_ESP()
display.show('1')
while not at_wifi_connected():
    try:
        at_wifi_connect(wifi_ssid, wifi_password)
    except Exception as exc:
        display.show(at_errorcode())
        sleep(2000)
        at_init_ESP()
        display.show('1')
display.show('2')

mac_address = at_get_mac_address()
log('My mac-addr: ' + mac_address)

at_mqtt_set_userconfig(1, 'gw-'+mac_address, mqtt_user, mqtt_password)
display.show('3')
    
while True:
    if not at_mqtt_connected():
        if not at_wifi_connected():
            reset()
        at_mqtt_connect(mqtt_host, mqtt_port, 1)
        at_set_on_mqtt_message(on_mqtt_message)
        at_mqtt_subscribe('microbit/input', 0)
        display('M')
    
    if button_a.was_pressed():
        if at_mqtt_connected():
            at_mqtt_publish('microbit/output', 'button A', 0, False)

    at_check_input()
    